# Project on Heart Disease (Group 73)

## Introduction

Heart disease is a broad term that refers to different heart conditions. Something in common is their effect on blood flow and dilation of veins to the heart. Other factors such as age, sex, and exercise are factors that are closely linked with heart disease. In our directory there are four data bases: Cleveland, Hungarian, Long Beach (California), and Switzerland. We decided to choose Long Beach due to similar culture (such as food and lifestyle) between Vancouver and Long beach. 


#test 1 with avery

#Prav Cells Be
low (helps out with reducing errors)

The relationship between these factors and their effect on predicting heart disease can be further examined through our research question: 
This project aims to provide a classification model which will predict whether a person has heart disease based on the factors of 

 1. Cleveland Clinic Foundation (cleveland.data)
     2. Hungarian Institute of Cardiology, Budapest (hungarian.data)
     3. V.A. Medical Center, Long Beach, CA (long-beach-va.data)
     4. University Hospital, Zurich, Switzerland (switzerland.data)


Cleveland Clinic Foundation (cleveland.data)
     2. Hungarian Institute of Cardiology, Budapest (hungarian.data)
     3. V.A. Medical Center, Long Beach, CA (long-beach-va.data)
     4. University Hospital, Zurich, Switzerland (switzerland.data)


This project aims to build a classification model to predict whether a patient presenting to the hospital would have a risk of heart disease by taking into account results from multiple medical tests. 
The dataset (https://archive.ics.uci.edu/ml/datasets/Heart+Disease) provides us with multiple attributes such as age and sex as well as a categorical attribute of whether a patient is in a risk of having a heart attack

The term “heart disease” refers to several types of heart conditions. The most common type of heart disease in the United States is coronary artery disease (CAD), which affects the blood flow to the heart. Decreased blood flow can cause a heart attack.


## Data Preparation

In [1]:
#Adding in librarys
library(tidyverse)
library(tidymodels)


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [2]:
#loading dataset from database

original <- "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/long-beach-va.data"
url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.va.data"
longbeach_data <- read_csv(url, col_names = FALSE)

Rows: 200 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (9): X4, X5, X6, X8, X9, X10, X11, X12, X13
dbl (5): X1, X2, X3, X7, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
#renaming data frame's column names accordingly
longbeach_set <- rename(longbeach_data,
    age = X1, 
    sex = X2, 
    cp = X3,
    trestbps = X4, 
    chol = X5, 
    fbs = X6,
    restecg = X7,
    thalach = X8, 
    exang = X9, 
    oldpeak = X10,                        
    slope = X11, 
    ca = X12,
    thal = X13, 
    num = X14)

In [4]:
#taking our most important columns
#then changing num to factor
longbeach_select <- longbeach_set |>
    select(age, chol, thalach, trestbps, oldpeak, num) |>
    mutate(num = as.factor(num))

To clean up the data, we first renamed all the columns accordingly to all the given attributes of the dataset. Then we took out all the unused attributes and changed variable num to factor because it is a categorical statical variable.

In [49]:
# split data into training set and testing set
longbeach_split <- initial_split(longbeach_select_final, prop=0.75, strata=num)
longbeach_train <- training(longbeach_split)
longbeach_test <- testing(longbeach_split)

We then split the cleaned data into training set and testing set, where 75% of the data are used to training while the remaining 25% will be used for testing. The proportion are chosen so that there is sufficient data to train the model with sufficient data to test the model and obtain the accuracy of the model. 

## Method

We will build the classification model using k-nearest neighbors. We will build it using the training data type longbeach_train and test it for accuracy using the testing data type longbeach_test. The number of k we use will be tuned to have the highest accuracy through cross-validation. The predictor variables we use will only be variables that have double as their data type and not factor data type. This is because factor data types can not be used for k-nearest neighbors as distance can not be calculated for factors. 

We can visualize the tuning using cross-validation by plotting a graph with number of k vs accuracy.


## Expected Outcomes and Significance

From this research project, we expect to find a correlation between risk factors (specifically, a person's age, cholesterol, resting heart rate, resting blood pressure, and ST depression) and the overall likelyhood that they have heart disease. With this research, we hope to create a model that can predict (to a high level of accuracy) whether a person has heart disease. We will use data taken from the UCI Machine Learning Repository's Heart Disease Dataset to train our model to predict whether someone has heart disease based on the 5 risk factors listed earlier. If our model is successful, it will be a great aid to the medical community by creating fast and easy diagnosis of heart disease in patients, and will allow doctors to quickly assist those in need. A doctor could simply input the measurements taken for each of the 5 risk factors and the model will determine whether the patient is developing heart disease or not. 

After our model is complete, future questions may be raised on this topic such as:
- Are there other diseases that can be accurately predicted based on a chosen set of measurable risk factors?
- Will it ever be ethical to nearly remove a doctor from the situation and trust a model instead?
- Is it possible to create a model that is 100% accurate?
- How might a model, such as ours, affect the speed at which heart disease is detected and treated, and could this lead to more lives saved in the future?

In [48]:
longbeach_select1 <- longbeach_select |>
filter(age != "?") |>
filter(chol != "?") |>
filter(thalach != "?") |>
filter(trestbps != "?") |>
filter(oldpeak != "?") |>
filter(num != "?")
#longbeach_select1

longbeach_select_final <- longbeach_select1 |>
mutate(chol = as.double(chol)) |>
mutate(thalach = as.double(thalach)) |>
mutate(trestbps = as.double(trestbps)) |>
mutate(oldpeak = as.double(oldpeak))

longbeach_select_final |>
print(n=10)

# A tibble: 137 × 6
     age  chol thalach trestbps oldpeak num  
   <dbl> <dbl>   <dbl>    <dbl>   <dbl> <fct>
 1    63   260     112      140     3   2    
 2    44   209     127      130     0   0    
 3    60   218     140      132     1.5 2    
 4    55   228     149      142     2.5 1    
 5    66   213      99      110     1.3 0    
 6    66     0     120      120    -0.5 0    
 7    65   236     105      150     0   3    
 8    60     0     140      180     1.5 0    
 9    60     0     141      120     2   3    
10    60   267     157      160     0.5 1    
# … with 127 more rows


In [53]:
#YAY we got our final data
#now we need to split into training and testing

#longbeach_train
#longbeach_test

long_beach_recipe <- recipe(num ~ age + chol + thalach + trestbps + oldpeak, data = longbeach_select_final) |>
   step_scale(all_predictors()) |>
   step_center(all_predictors())

long_beach_recipe

Recipe

Inputs:

      role #variables
   outcome          1
 predictor          5

Operations:

Scaling for all_predictors()
Centering for all_predictors()

In [101]:
# Now we need to make KNN spec and workflow

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 21) |>
      set_engine("kknn") |>
      set_mode("classification")
#knn_spec
long_beach_fit <- workflow() |>
      add_recipe(long_beach_recipe) |>
      add_model(knn_spec) |>
      fit(data = longbeach_train)
#long_beach_fit

In [102]:
long_beach_predictions <- predict(long_beach_fit , longbeach_test) |>
      bind_cols(longbeach_test)
long_beach_predictions |>
print(n=10)

# A tibble: 36 × 7
   .pred_class   age  chol thalach trestbps oldpeak num  
   <fct>       <dbl> <dbl>   <dbl>    <dbl>   <dbl> <fct>
 1 3              63   260     112      140     3   2    
 2 2              55   228     149      142     2.5 1    
 3 1              66   213      99      110     1.3 0    
 4 2              65   236     105      150     0   3    
 5 2              59     0     117      140     1   1    
 6 1              62     0     120      110     0.5 1    
 7 2              63     0     120      126     1.5 0    
 8 2              64     0     122      144     1   3    
 9 0              63     0      84      136     0   2    
10 2              55     0     125      120     2.5 1    
# … with 26 more rows


In [103]:
#Now we need to find accuracy

long_beach__prediction_accuracy <- long_beach_predictions |> #df
        metrics(truth = num, estimate = .pred_class)   #real vs estimations

long_beach__prediction_accuracy



.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,multiclass,0.3611111
kap,multiclass,0.1375000
